# Sheath Expansion Simulations Results - Prototyping the Analysis of vers 2.14
First from a series of notebooks analysing the sheath expansions simulations in a manner similar to Bergmann in his 2002 paper. This one prototypes the analysis procedure - initially formulated in restart_test/recompilation_restart_comparison.ipynb - to be implemented in flopter.

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
import configparser
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
# import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [1]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

NameError: name 'pth' is not defined

In [3]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [4]:
skippable_scans = set() 
single_sims = set()

In [5]:
scans_searchstr = '*/*/sheath_exp/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        print(f'Single sim {scan} found')
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
    print(f'{scan}: {all_run_dirs[scan]}\n')


marconi/spice2/sheath_exp/semi-angled_recessed: ['marconi/spice2/sheath_exp/semi-angled_recessed/alpha_yz_-10.0', 'marconi/spice2/sheath_exp/semi-angled_recessed/alpha_yz_-2.0', 'marconi/spice2/sheath_exp/semi-angled_recessed/alpha_yz_-3.0', 'marconi/spice2/sheath_exp/semi-angled_recessed/alpha_yz_-30.0', 'marconi/spice2/sheath_exp/semi-angled_recessed/alpha_yz_-5.0']

marconi/spice2/sheath_exp/semi-angled_1: ['marconi/spice2/sheath_exp/semi-angled_1/alpha_yz_-1.0']

marconi/spice2/sheath_exp/angled_recessed: ['marconi/spice2/sheath_exp/angled_recessed/alpha_yz_-10.0', 'marconi/spice2/sheath_exp/angled_recessed/alpha_yz_-2.0', 'marconi/spice2/sheath_exp/angled_recessed/alpha_yz_-3.0', 'marconi/spice2/sheath_exp/angled_recessed/alpha_yz_-30.0', 'marconi/spice2/sheath_exp/angled_recessed/alpha_yz_-5.0']

marconi/spice2/sheath_exp/flat_semi-recessed_1: ['marconi/spice2/sheath_exp/flat_semi-recessed_1/alpha_yz_-1.0']

marconi/spice2/sheath_exp/angled_recessed_1: ['marconi/spice2/sheath_exp

In [6]:
# list(spice_dir.glob('*/*/sheath_exp/*'))

In [7]:
skippable = {}
scans = list(scans)
scans.sort()

In [8]:
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/spice2/sheath_exp/angled
1 marconi/spice2/sheath_exp/angled_1
2 marconi/spice2/sheath_exp/angled_recessed
3 marconi/spice2/sheath_exp/angled_recessed_1
4 marconi/spice2/sheath_exp/angled_semi-recessed
5 marconi/spice2/sheath_exp/angled_semi-recessed_1
6 marconi/spice2/sheath_exp/flat
7 marconi/spice2/sheath_exp/flat_recessed
8 marconi/spice2/sheath_exp/flat_semi-recessed
9 marconi/spice2/sheath_exp/flat_semi-recessed_1
10 marconi/spice2/sheath_exp/semi-angled
11 marconi/spice2/sheath_exp/semi-angled_1
12 marconi/spice2/sheath_exp/semi-angled_recessed
13 marconi/spice2/sheath_exp/semi-angled_recessed_1
14 marconi/spice2/sheath_exp/semi-angled_semi-recessed
15 marconi/spice2/sheath_exp/semi-angled_semi-recessed_1


---
## Generalising the Groupby Method of Sweep Extraction

In [9]:
spl_path = spice_dir / all_run_dirs[scans[1]][0]
print(spl_path)

/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled_1/alpha_yz_-1.0


---
Check if we can extract the spice version from the simulation directory.

In [15]:
[p.name for p in spl_path.glob('*.*')]


['alpha_yz_-1.0.mat',
 'input.inp',
 'jobs.txt',
 'log.err',
 'log.ongoing.err',
 'log.ongoing.out',
 'log.out',
 't-alpha_yz_-1.0.mat']

Answer: not easily. Will have to feed version in as a kwarg :( 

---

In [31]:
import importlib
importlib.reload(spl)

<module 'flopter.spice.splopter' from '/home/jleland/coding/projects/flopter/flopter/spice/splopter.py'>

In [23]:
splopter = spl.Splopter(spl_path, reduce=desired_variables, ignore_tzero_fl=True)
splopter.prepare(denormaliser_fl=False, 
                 homogenise_fl=True, 
                 find_se_temp_fl=False, 
                 backup_concat_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled_1/alpha_yz_-1.0) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [11]:
# list(splopter.tdata.t_dict.keys())

Had to turn off the t-zeroing detection as it seems to have occurred but not broken anything in this file. 

Now to plot to check a couple of things. 

In [15]:
fig, ax = plt.subplots(2, sharey=True)

ax[0].plot(splopter.raw_data['I'])
ax[1].plot(splopter.raw_data['V'])

The voltage looks broken as it's prepending when it doesn't need to. Seeing as the format for homogenisation is quite different for this version of SPICE (2.14), I could split up the homogenisation procedure into two different objects (or some similar differentiation)

In [16]:
fig, ax = plt.subplots(2, sharex=True)

t, V, I_e, I_i = splopter.get_tdata_raw_iv(splopter.tdata)

ax[0].plot(V)
ax[1].plot(I_e + I_i)

In [24]:
print(f'raw_data["I"]: \t {len(splopter.raw_data["I"])}')
print(f'raw_data["V"]: \t {len(splopter.raw_data["V"])} \n')

print(f'tdata.t: \t {len(splopter.tdata.t)}')
print(f'ProbePot: \t {len(splopter.tdata.t_dict["ProbePot"])} \n')

print(f'V: \t {len(V)}')
print(f'I_i: \t {len(I_e)} \n')

print(f'ta: \t {splopter.parser["geom"]["ta"]}')
print(f'tp: \t {splopter.parser["geom"]["tp"]}')
print(f'tc: \t {splopter.parser["geom"]["tc"]} \n')

raw_data["I"]: 	 9113580
raw_data["V"]: 	 9113580 

tdata.t: 	 1
ProbePot: 	 9113581 

V: 	 9113581
I_i: 	 9113580 

ta: 	 3.0
tp: 	 4.0
tc: 	 3.0 



In [25]:
ta = splopter.parser.getfloat('geom', 'ta')
tp = splopter.parser.getfloat('geom', 'tp')
r = ta/tp
r

0.75

In [26]:
r * len(I_e)

6835185.0

In [29]:
splopter.parser.get_sweep_length(len(I_e), V)
# list(splopter.parser.keys())
#.get_probe_obj_indices()

6835185

In [17]:
fig, ax = plt.subplots(2)

sweep_slc = slice(math.ceil(r * len(I_e)), len(I_e))
index = np.arange(0, len(V))
pot = np.squeeze(V)

print(f'index.shape = {index.shape}')
print(f'pot.shape = {pot.shape}')
print(f'current.shape = {splopter.raw_data["I"].shape}')

ax[0].plot(index[:-1], splopter.raw_data['I'])
ax[0].plot(index[sweep_slc], splopter.raw_data['I'][sweep_slc], color='tab:green')

pot_ax = ax[0].twinx()
pot_ax.plot(index, pot, color='tab:orange')
pot_ax.plot(index[sweep_slc], pot[sweep_slc], color='tab:red')


ax[1].plot(np.squeeze(splopter.tdata.t_dict['ProbePot'])[sweep_slc], splopter.raw_data['I'][sweep_slc])

index.shape = (915956,)
pot.shape = (915956,)
current.shape = (915955,)


## Make Dataframe
The sweep can be made into a dataframe for ease of grouping and averaging into a smoothly varying Voltage waveform. 

Some checks are subsequently made that do not need to be included in the splopter implementation.

In [18]:
df = pd.DataFrame({'voltage':V[sweep_slc], 'current': (I_e + I_i)[sweep_slc]})
df

,voltage,current
0,-10.00,39.0
1,-9.95,31.0
2,-9.95,42.0
3,-9.95,30.0
4,-9.95,27.0
...,...,...
228983,10.05,0.0
228984,10.05,0.0
228985,10.05,0.0
228986,10.05,0.0


In [19]:
voltage_groups = df.groupby('voltage').groups

In [20]:
sizes = []
for voltage_group in voltage_groups:
    voltage_group_size = len(voltage_groups[voltage_group])
    sizes.append(voltage_group_size)
    if voltage_group_size > 5:
        print(voltage_group, voltage_group_size)

-9.950000000000001 572
-9.9 572
-9.85 572
-9.799999999999999 572
-9.75 572
-9.700000000000001 572
-9.65 572
-9.6 572
-9.549999999999999 572
-9.5 572
-9.450000000000001 572
-9.4 572
-9.35 572
-9.299999999999999 572
-9.25 572
-9.200000000000001 572
-9.15 572
-9.1 572
-9.049999999999999 572
-9.0 572
-8.950000000000001 572
-8.9 572
-8.85 572
-8.799999999999999 572
-8.75 572
-8.700000000000001 572
-8.65 572
-8.6 572
-8.549999999999999 572
-8.5 572
-8.450000000000001 572
-8.4 572
-8.35 572
-8.299999999999999 572
-8.25 572
-8.200000000000001 572
-8.15 572
-8.1 572
-8.049999999999999 572
-8.0 572
-7.950000000000001 572
-7.8999999999999995 572
-7.8500000000000005 572
-7.799999999999999 572
-7.75 572
-7.700000000000001 572
-7.6499999999999995 572
-7.6000000000000005 572
-7.549999999999999 572
-7.5 572
-7.450000000000001 572
-7.3999999999999995 572
-7.3500000000000005 572
-7.299999999999999 572
-7.25 572
-7.200000000000001 572
-7.1499999999999995 572
-7.1000000000000005 572
-7.049999999999999 572

In [21]:
voltage_val_oi = voltage_groups[-9.0]
voltage_val_oi

Int64Index([10869, 10870, 10871, 10872, 10873, 10874, 10875, 10876, 10877,
            10878,
            ...
            11431, 11432, 11433, 11434, 11435, 11436, 11437, 11438, 11439,
            11440],
           dtype='int64', length=572)

In [22]:
def plot_and_fit_gaussian(data):
    gaussian = fts.NormalisedGaussianFitter()
    fig, ax = plt.subplots()
    hist, bin_edges = np.histogram(data, bins='auto', density=True)
    bins = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax.plot(bins, hist)
    
    params = [np.std(data), np.mean(data)]
    fit_data = gaussian.fit(bins, hist, initial_vals=params) 
    x = np.linspace(min(bins), max(bins), 1000)
    ax.plot(x, fit_data.fit_function(x), label=r'Gaussian fit - $\sigma$ = {:.3g}'.format(fit_data.get_param("sigma")))

    ax.legend()
    
    return fit_data

plot_and_fit_gaussian(df.iloc[voltage_val_oi]['current'])
df.iloc[voltage_val_oi]['current'].plot()

In [24]:
bin_size = int(np.median(sizes))
red_bin_size = int(0.1 * bin_size)

### Group and average over individual voltage values

In [25]:
red_smoothed_df = df.groupby('voltage').filter(lambda x: len(x) == bin_size).groupby('voltage').tail(red_bin_size).groupby('voltage').mean()
red_smoothed_df.plot()

In [26]:
filtered_df = df.groupby('voltage').filter(lambda x: len(x) == bin_size)
smoothed_df = filtered_df.groupby('voltage').mean()
errors = filtered_df.groupby('voltage').std()
smoothed_df

,current
voltage,
-9.95,37.690546
-9.90,37.494743
-9.85,37.402080
-9.80,37.150333
-9.75,37.358375
...,...
9.80,-288.795499
9.85,-289.398582
9.90,-270.613523


Comparison of whether, as per Michael's suggestion, using the latter 10% of each bin makes a difference in eliminating oscillations caused by the step change in voltage. 

In [27]:
fig, ax = plt.subplots()

red_smoothed_df.plot(ax=ax)
smoothed_df.plot(ax=ax)

A: Not a whole lot in this case, makes the smoothing a bit worse though - especially in the electron region. Keep in the option of doing it (as it's so easy) but make it non-default behaviour. 

In [48]:
smoothed_df['d_current'] = errors['current'].values / np.sqrt(bin_size)
smoothed_df['time'] = np.arange(0, smoothed_df['current'].size) * np.squeeze(splopter.tdata.dt) * bin_size

In [35]:
print(splopter.parser.get_commented_params())
print(splopter.parser['mks']['mks_n0'])
print(splopter.parser['mks']['mks_b'])
print(splopter.parser['mks']['mks_te'])

{'n_e': 1e+18, 'T_e': 5.0, 'B': 0.8, 'L_probe': 5.0}
1e+18
0.8
5


In [18]:
dV = splopter.parser.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = splopter.parser.getfloat('mks', 'mks_te')
n_0 = splopter.parser.getfloat('mks', 'mks_n0')

In [40]:
smoothed_df = smoothed_df.reset_index()

In [45]:
smoothed_df['corr_voltage'] = smoothed_df['voltage'] * dV

In [42]:
smoothed_df['current'] = -smoothed_df['current']

In [46]:
smoothed_df

,voltage,current,corr_voltage
0,-9.95,-37.690546,-49.75
1,-9.90,-37.494743,-49.50
2,-9.85,-37.402080,-49.25
3,-9.80,-37.150333,-49.00
4,-9.75,-37.358375,-48.75
...,...,...,...
395,9.80,288.795499,49.00
396,9.85,289.398582,49.25
397,9.90,270.613523,49.50
398,9.95,255.794485,49.75


In [49]:
iv_data = iv.IVData.from_dataset(smoothed_df)

In [50]:
iv_data.plot()

<ErrorbarContainer object of 3 artists>

In [96]:
fit_data = iv_data.multi_fit(sat_region=-4, plot_fl=True)

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  return self.func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  func(*args)


In [95]:
a = fit_data.get_sheath_exp()


NameError: name 'fit_data' is not defined

In [87]:
L = splopter.parser.getfloat('rectangle2', 'yhigh') - splopter.parser.getfloat('rectangle2', 'ylow')
lambda_D = lpu.debye_length(T_e, n_0)
theta = np.abs(np.squeeze(splopter.tdata.alphayz))
I_0 = c.ELEM_CHARGE * n_0 * lpu.sound_speed(T_e) * L * lambda_D * np.sin(theta)

print(f"L = {L} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta = {theta} \n"
      f"I_0 = {I_0}")

L = 300.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta = 0.17453292519943295 
I_0 = 4.294003089384486


In [88]:
for shape in ['rectangle0', 'rectangle1', 'rectangle2']:
    print(f"{shape} = {splopter.parser[shape]['name']}:")
    for coord in ['y', 'z']:
        print(f"\t{coord}low: {splopter.parser[shape][coord+'low']}")
        print(f"\t{coord}high: {splopter.parser[shape][coord+'high']}")

rectangle0 = 'forewall':
	ylow: 0
	yhigh: 68
	zlow: 0
	zhigh: 94
rectangle1 = 'rearwall':
	ylow: 488
	yhigh: 556
	zlow: 0
	zhigh: 94
rectangle2 = 'probe':
	ylow: 128
	yhigh: 428
	zlow: 0
	zhigh: 94


In [94]:
fig, ax = plt.subplots()

ax.plot(smoothed_df['corr_voltage'], smoothed_df['current'] / I_0)
ax.set_xlim(-30, 0)
ax.set_ylim(-15, 85)
# ax.set_yscale('log')

(-15, 85)

In [ ]:
fig, ax = plt.subplots()

ax.plot(smoothed_df['corr_voltage'], smoothed_df['current'] / I_0)
ax.set_xlim(-30, 0)
ax.set_ylim(-15, 85)
# ax.set_yscale('log')

# Post implementation check

In [30]:
splopter_old = spl.Splopter(spl_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.13)
splopter_old.prepare(denormaliser_fl=False, 
                     homogenise_fl=True, 
                     find_se_temp_fl=False, 
                     backup_concat_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled_1/alpha_yz_-1.0) doesn't seem to be valid.
Continuing anyway.


In [33]:
importlib.reload(iv)
importlib.reload(spl)

<module 'flopter.spice.splopter' from '/home/jleland/coding/projects/flopter/flopter/spice/splopter.py'>

In [10]:
splopter = spl.Splopter(spl_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14)
splopter.prepare(denormaliser_fl=False, 
                 homogenise_fl=True, 
                 find_se_temp_fl=False, 
                 backup_concat_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled_1/alpha_yz_-1.0) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [12]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(splopter.raw_data['V'])
ax[1].plot(splopter.raw_data['I'])

ax[0].plot(splopter_old.raw_data['V'])
ax[1].plot(splopter_old.raw_data['I'])

NameError: name 'splopter_old' is not defined

In [37]:
fig, ax = plt.subplots(2, 2, sharey=True)

ax[0,0].plot(splopter.iv_data['V'])
ax[1,0].plot(splopter.iv_data['I'])
ax[1,0].set_xlabel('New')

ax[0,1].plot(splopter_old.iv_data['V'])
ax[1,1].plot(splopter_old.iv_data['I'])
ax[1,1].set_xlabel('Old')

Text(0.5, 0, 'Old')

Success!

## Make a function of post-homogenisation analysis

In [13]:
iv_data = splopter.iv_data

In [14]:
fit_data = iv_data.multi_fit(sat_region=-4, plot_fl=True)

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  return self.func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  func(*args)


In [15]:
a = fit_data.get_sheath_exp()
